In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv1D
from tensorflow.keras import Model

from boto3_utilities import *

In [30]:
df_rest = read_s3_file("sensor-data-keisuke-nakata", "sample=256/label=rest/latest/data.csv")
df_rest.head()

,dt,grp,cnt,sample,emg,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label
0,2021/10/12 16:31:21,0,0,256,3.000978,0.012085,0.033325,0.980957,1.770020,1.281738,0.854492,rest
1,2021/10/12 16:31:21,0,1,256,3.093842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,rest
2,2021/10/12 16:31:21,0,2,256,3.010753,0.013550,0.033447,0.979492,1.525879,1.464844,0.854492,rest
3,2021/10/12 16:31:21,0,3,256,3.069404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,rest
4,2021/10/12 16:31:21,0,4,256,3.005865,0.013184,0.033203,0.979980,1.586914,1.342773,0.854492,rest


In [7]:
df_touch = read_s3_file("sensor-data-keisuke-nakata", "sample=256/label=touch/latest/data.csv")
df_touch.head()

,dt,grp,cnt,sample,emg,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label
0,2021/10/12 16:15:49,0,0,256,3.866080,0.021851,0.034668,0.978027,1.647949,1.708984,0.915527,touch
1,2021/10/12 16:15:49,0,1,256,3.768328,0.020508,0.036987,0.974365,1.831055,1.525879,0.915527,touch
2,2021/10/12 16:15:49,0,2,256,3.841642,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,touch
3,2021/10/12 16:15:49,0,3,256,3.817204,0.020630,0.035156,0.974365,2.014160,1.342773,0.976562,touch
4,2021/10/12 16:15:49,0,4,256,3.778104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,touch


In [125]:
rest_list = df_rest.groupby("grp")["emg"].apply(list)
df_rest_emg = pd.DataFrame(rest_list).reset_index()['emg']

df_rest_emg = pd.DataFrame(df_rest_emg.values.tolist())
df_rest_emg.columns = [f'{i}' for i in range(0, 256)]
df_rest_emg['label'] = 'rest'

df_rest_emg.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,label
0,3.000978,3.093842,3.010753,3.069404,3.005865,3.079179,3.030303,2.981427,3.088954,3.005865,...,3.035191,3.064516,3.010753,3.098729,3.088954,3.040078,3.079179,3.010753,3.084066,rest
1,4.584555,4.628543,4.613881,4.589443,4.608993,4.594330,4.506354,4.579668,4.393939,4.545455,...,3.929619,3.797654,3.724340,3.528837,3.543499,3.607038,3.582600,3.621701,3.479961,rest
2,3.235582,3.054741,3.172043,3.103617,3.079179,3.118279,3.059628,3.049853,3.020528,3.069404,...,3.079179,2.991202,3.049853,3.069404,2.991202,3.064516,3.010753,2.996090,3.040078,rest
3,3.064516,2.908113,2.908113,3.074291,2.996090,3.025415,3.103617,3.035191,3.084066,3.049853,...,3.054741,3.074291,3.005865,3.123167,3.098729,3.084066,3.255132,3.132942,3.176931,rest
4,3.123167,3.181818,3.157380,3.113392,3.172043,3.054741,3.167156,3.137830,3.069404,3.181818,...,3.103617,3.074291,3.123167,3.064516,3.118279,3.118279,3.079179,3.118279,3.079179,rest


In [126]:
touch_list = df_touch.groupby("grp")["emg"].apply(list)
df_touch_emg = pd.DataFrame(touch_list).reset_index()['emg']

df_touch_emg = pd.DataFrame(df_touch_emg.values.tolist())
df_touch_emg.columns = [f'{i}' for i in range(0, 256)]
df_touch_emg['label'] = 'touch'

df_touch_emg.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,label
0,3.866080,3.768328,3.841642,3.817204,3.778104,3.822092,3.714565,3.831867,3.778104,3.768328,...,3.665689,3.616813,3.563050,3.690127,3.567937,3.641251,3.651026,3.607038,3.636364,touch
1,3.636364,3.714565,3.651026,3.651026,3.690127,3.616813,3.719453,3.651026,3.670577,3.724340,...,3.880743,3.973607,3.954057,3.880743,3.973607,3.880743,3.949169,3.929619,3.866080,touch
2,3.954057,3.831867,3.910069,3.890518,3.861192,3.905181,3.822092,3.958944,3.939394,3.880743,...,3.988270,3.949169,3.856305,3.968719,3.870968,3.910069,3.914956,3.890518,3.998045,touch
3,3.245357,3.123167,3.186706,3.396872,3.421310,3.499511,3.421310,3.318671,3.504399,3.445748,...,2.922776,3.015640,2.956989,2.913001,2.961877,2.883676,2.976540,2.971652,2.932551,touch
4,3.651026,3.553275,3.597263,3.558162,3.680352,3.660802,3.602150,3.812317,3.631476,3.714565,...,3.817204,3.724340,3.807429,3.768328,3.753666,3.807429,3.719453,3.802541,3.748778,touch


In [ ]:
model = models.Sequential()
model.add(layers.Conv1D(16, (3, 3), activation='relu', input_shape=(120, 40, 1)))
tf.keras.layers.BatchNormalization(
    axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,
    beta_initializer='zeros', gamma_initializer='ones',
    moving_mean_initializer='zeros',
    moving_variance_initializer='ones', beta_regularizer=None,
    gamma_regularizer=None, beta_constraint=None, gamma_constraint=None, **kwargs
)

model.add(layers.Conv1D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))        # 犬猫と同じ2クラス分類で

# モデルのコンパイル
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])